In [ ]:
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import yfinance as yf

In [ ]:
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [ ]:
df = pd.read_parquet("../data/data.parquet")
# events = pd.read_parquet("../data/events.parquet")
# df

In [ ]:

def fib_retracement(data: pd.DataFrame, high: str, low: str, target_column: str, levels: list = [0.236, 0.382, 0.618, 1.0]):
    for level in levels:
        data[f"{target_column}_fib_{level}"] = data[high] - (data[high] - data[low]) * level

    return data


def create_lag_features(data: pd.DataFrame, target_column, lag_steps=1):
    if isinstance(lag_steps, int):
        for i in range(1, lag_steps + 1):
            data[f"{target_column}_lag_{i}"] = data[target_column].shift(i)

    if isinstance(lag_steps, list):
        for i in lag_steps:
            data[f"{target_column}_lag_{i}"] = data[target_column].shift(i)

    return data


def simple_moving_average(data: pd.DataFrame, target_column: str | list, window_size: int | list = 3):
    if isinstance(target_column, list) and isinstance(window_size, list):
        for col in target_column:
            for window in window_size:
                data[f"{col}_rolling_mean_{window}"] = data[col].rolling(window=window).mean()

    elif isinstance(target_column, str):
        data[f"{target_column}_rolling_mean_{window_size}"] = data[target_column].rolling(window=window_size).mean()

    elif isinstance(target_column, list):
        for col in target_column:
            data[f"{col}_rolling_mean_{window_size}"] = data[col].rolling(window=window_size).mean()

    return data


def exponential_moving_average(data: pd.DataFrame, target_column: str | list, window_size: int | list = 3):
    if isinstance(target_column, str):
        target_column = [target_column]
    if isinstance(window_size, int):
        window_size = [window_size]

    for col in target_column:
        for window in window_size:
            data[f"{col}_rolling_exp_mean_{window}"] = data[col].ewm(span=window).mean()

    return data


def bollinger_bands(data: pd.DataFrame, target_column: str, window_size: int = 20):
    data[f"{target_column}_rolling_mean_{window_size}"] = data[target_column].rolling(window=window_size).mean()
    data[f"{target_column}_rolling_std_{window_size}"] = data[target_column].rolling(window=window_size).std()
    data[f"{target_column}_bollinger_upper_{window_size}"] = data[f"{target_column}_rolling_mean_{window_size}"] + 2 * data[
        f"{target_column}_rolling_std_{window_size}"]
    data[f"{target_column}_bollinger_lower_{window_size}"] = data[f"{target_column}_rolling_mean_{window_size}"] - 2 * data[
        f"{target_column}_rolling_std_{window_size}"]

    return data

def greater_than(data: pd.DataFrame, src_column: str, target_column: str):
    """
    Check if the src_column is greater then target_column
    Return 1 if src_column is greater then target_column and 0 otherwise
    """
    data[f"{src_column}_greater_than_{target_column}"] = np.where(data[src_column] > data[target_column], 1, 0)

    return data


        


# def plot_bollinger_bands(data: pd.DataFrame, target_column: str, window_size: int = 20):
#     data[f"{target_column}_rolling_mean_{window_size}"] = data[target_column].rolling(window=window_size).mean()
#     data[f"{target_column}_rolling_std_{window_size}"] = data[target_column].rolling(window=window_size).std()
#     data[f"{target_column}_bollinger_upper_{window_size}"] = data[f"{target_column}_rolling_mean_{window_size}"] + 2 * data[
#         f"{target_column}_rolling_std_{window_size}"]
#     data[f"{target_column}_bollinger_lower_{window_size}"] = data[f"{target_column}_rolling_mean_{window_size}"] - 2 * data[
#         f"{target_column}_rolling_std_{window_size}"]

#     plt.figure(figsize=(12, 6))
#     plt.plot(data[target_column], label="Close Price", color="blue")
#     plt.plot(data[f"{target_column}_rolling_mean_{window_size}"], label="Rolling Mean", color="red")
#     plt.plot(data[f"{target_column}_bollinger_upper_{window_size}"], label="Bollinger Upper", color="green")
#     plt.plot(data[f"{target_column}_bollinger_lower_{window_size}"], label="Bollinger Lower", color="green")
#     plt.title(f"{target_column} Bollinger Bands")
#     plt.legend()
#     plt.show()

#     return data


def average_true_range(data: pd.DataFrame, window_size: int = 14):
    data[f"high_low_{window_size}"] = data["high"] - data["low"]
    data[f"high_close_{window_size}"] = np.abs(data["high"] - data["close"].shift(1))
    data[f"low_close_{window_size}"] = np.abs(data["low"] - data["close"].shift(1))
    data[f"true_range_{window_size}"] = np.max(
        [data[f"high_low_{window_size}"], data[f"high_close_{window_size}"], data[f"low_close_{window_size}"]], axis=0)
    data[f"average_true_range_{window_size}"] = data[f"true_range_{window_size}"].rolling(window=window_size).mean()

    return data


def relative_strength_index(data: pd.DataFrame, column: str = "close", window_size: int = 14):
    delta = data[column].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window_size).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window_size).mean()
    rs = gain / loss
    data[f"rsi_{window_size}"] = 100 - (100 / (1 + rs))

    return data


def macd(data: pd.DataFrame, column: str = "close", short_window: int = 12, long_window: int = 26, signal_window: int = 9):
    data[f"short_ema_{short_window}"] = data[column].ewm(span=short_window, adjust=False).mean()
    data[f"long_ema_{long_window}"] = data[column].ewm(span=long_window, adjust=False).mean()
    data[f"macd_{short_window}_{long_window}"] = data[f"short_ema_{short_window}"] - data[f"long_ema_{long_window}"]
    data[f"signal_{signal_window}"] = data[f"macd_{short_window}_{
        long_window}"].ewm(span=signal_window, adjust=False).mean()
    data[f"macd_hist_{short_window}_{long_window}_{signal_window}"] = data[f"macd_{
        short_window}_{long_window}"] - data[f"signal_{signal_window}"]

    return data


def stochastic_oscillator(data: pd.DataFrame, window_size: int = 14):
    data[f"stochastic_oscillator_{window_size}"] = (data["close"] - data["low"].rolling(window=window_size).min()) / (
        data["high"].rolling(window=window_size).max() - data["low"].rolling(window=window_size).min())

    return data


def williams_r(data: pd.DataFrame, window_size: int = 14):
    data[f"williams_r_{window_size}"] = (data["high"].rolling(window=window_size).max() - data["close"]) / (
        data["high"].rolling(window=window_size).max() - data["low"].rolling(window=window_size).min())

    return data


def money_flow_index(data: pd.DataFrame, window_size: int = 14):
    typical_price = (data["high"] + data["low"] + data["close"]) / 3
    raw_money_flow = typical_price * data["volume"]
    money_flow_ratio = raw_money_flow.rolling(window=window_size).sum() / (typical_price * data["volume"]).rolling(
        window=window_size).sum()
    data[f"money_flow_index_{window_size}"] = 100 - (100 / (1 + money_flow_ratio))

    return data


def on_balance_volume(data: pd.DataFrame, offset: int = 1):
    data[f"on_balance_volume_{offset}"] = np.where(
        data["close"] > data["close"].shift(offset), data["volume"], -data["volume"])
    data[f"on_balance_volume_{offset}"] = data[f"on_balance_volume_{offset}"].cumsum()

    return data


def volume_weighted_average_price(data: pd.DataFrame, window_size: int = 14):
    data[f"volume_weighted_average_price_{window_size}"] = (
        data["close"] * data["volume"]).rolling(window=window_size).sum() / data["volume"].rolling(window=window_size).sum()

    return data


def volitility(data: pd.DataFrame, column: str = "close", window_size: int = 14):
    data[f"volitility_{column}_{window_size}"] = data[column].rolling(window=window_size).std()
    data[f"volitility_pct_change_{window_size}_{column}"] = data[column].pct_change().rolling(window=window_size).std()

    return data



def apply_technical_indicators(data: pd.DataFrame, target_column: str | list = "close", window_size: int | list = 14):

    if isinstance(target_column, str):
        target_column = [target_column]
    if isinstance(window_size, int):
        window_size = [window_size]

    for col in target_column:
        for window in window_size:
            data = volitility(data, col, window)
            data = average_true_range(data, window)
            data = simple_moving_average(data, col, window)
            data = exponential_moving_average(data, col, window)
            data = relative_strength_index(data, col, window)
            data = macd(data, col)
            data = stochastic_oscillator(data, window)
            data = williams_r(data, window)
            data = money_flow_index(data, window)
            data = on_balance_volume(data)
            data = volume_weighted_average_price(data, window)
            data = bollinger_bands(data, col, window)
            # data = fib_retracement(data, "high", "low", col)


    return data


def forward_fill_with_decay(df: pd.DataFrame, column, decay_factor):
    """
    Forward fills missing values in a specified column with a decay factor applied.

    Parameters:
    - df: The DataFrame containing the data.
    - column: The column name (or index) to apply forward fill with decay.
    - decay_factor: The factor by which the previous value decays (0 < decay_factor < 1).

    Returns:
    - The DataFrame with missing values filled with decayed forward fill.
    """
    if not 0 < decay_factor < 1:
        raise ValueError("Decay factor must be between 0 and 1.")

    df = df.copy()  # Avoid modifying the original DataFrame
    df[column] = df[column].astype(float)  # Ensure column is of float type for NaNs

    # Initialize variables
    last_value = np.nan
    decay = 1.0

    for i in range(len(df)):
        if pd.notna(df.at[i, column]):
            # Update the last value and reset decay
            last_value = df.at[i, column]
            decay = 1.0
            df.at[i, column] = last_value * decay
            decay *= decay_factor  # Apply the decay factor for the next value

        elif pd.isna(df.at[i, column]) and pd.notna(last_value):
            # Apply decay to the last value and fill the current cell
            df.at[i, column] = last_value * decay
            decay *= decay_factor  # Apply the decay factor for the next value

    return df


def apply_event(data: pd.DataFrame, event: pd.DataFrame, prefix_name, falloff: bool = True, decay_factor: float = 0.99) -> pd.DataFrame:
    """Join Events df to data df based on Date"""
    event = event.copy(deep=True)

    event["date"] = pd.to_datetime(event["date"])

    # Rename Events columns to event_* to avoid conflicts
    for column in event.columns:
        if column != "date":
            event.rename(columns={column: f"event_{column}"}, inplace=True)

    df = data.merge(event, on="date", how="left").copy(deep=True)

    df["event_decay"] = np.where(pd.notna(df["event_name"]), 1, np.nan)
    df["event_name"] = df["event_name"].ffill().infer_objects(copy=False)
    df["event_value"] = df["event_value"].ffill().infer_objects(copy=False)
    df["event_sentiment"] = df["event_sentiment"].ffill().infer_objects(copy=False)

    df = df.rename(columns={
        "event_name": f"{prefix_name}_event_name",
        "event_value": f"{prefix_name}_event_value",
        "event_sentiment": f"{prefix_name}_event_sentiment",
        "event_decay": f"{prefix_name}_event_decay"
    })

    # forward fill sentiment with decay
    if falloff:
        df = forward_fill_with_decay(df, f"{prefix_name}_event_decay", decay_factor)

    return df


def day_of_week(data: pd.DataFrame) -> pd.DataFrame:
    data["day_of_week"] = data["date"].dt.dayofweek.astype(int)
    return data


def day_of_month(data: pd.DataFrame) -> pd.DataFrame:
    data["day_of_month"] = data["date"].dt.day.astype(int)
    return data


def month(data: pd.DataFrame) -> pd.DataFrame:
    data["month"] = data["date"].dt.month.astype(int)
    return data


def year(data: pd.DataFrame) -> pd.DataFrame:
    data["year"] = data["date"].dt.year.astype(int)
    return data


def week_of_year(data: pd.DataFrame) -> pd.DataFrame:
    data["week_of_year"] = data["date"].dt.isocalendar().week.astype(int)
    return data


def is_not_nan_column_and_default(data: pd.DataFrame, column) -> pd.DataFrame:
    if isinstance(column, list):
        for col in column:
            data[f"is_nan_{col}"] = data[col].notna().astype(int)
            data[col] = data[col].fillna(0)
        return data

    if isinstance(column, str):
        data[f"is_nan_{column}"] = data[column].notna().astype(int)
        data[column] = data[column].fillna(0)
        return data


def future_value(data: pd.DataFrame, column: str, offset: int, drop_recent: bool = True) -> pd.DataFrame:
    data[f"target_{column}"] = data[column].shift(-offset)
    # Drop last X offset rows
    if drop_recent:  # Drop to remove NaNs
        data = drop_recent_with_offset(data, offset)

    return data


def drop_recent_with_offset(data: pd.DataFrame, offset: int) -> pd.DataFrame:
    return data[:-offset]


def percent_change_between_columns(data: pd.DataFrame, column: str, column2: str) -> pd.DataFrame:
    data[f"percent_change_{column}"] = (data[column2] - data[column]) / data[column]
    return data


def percent_change(data: pd.DataFrame, column: str, offset: int | list = 1, drop_recent: bool = True) -> pd.DataFrame:
    if isinstance(offset, int):
        offset = [offset]

    for off in offset:
        data[f"{column}_percent_change_{off}"] = (data[column].shift(-off) - data[column]) / data[column]

    if drop_recent:  # Drop to remove NaNs
        data = drop_recent_with_offset(data, max(offset))

    return data

def up_down(data: pd.DataFrame, column: str, offset: int, drop_recent: bool = True) -> pd.DataFrame:
    """
    Simply tells wheither the price went up or down in the last window_size days
        Returns 1 if the price went up, 0 if the price went down, and 0 if the price stayed the same
    """
    if isinstance(offset, int):
        offset = [offset]

    for off in offset:
        data[f"up_down_{column}_{off}"] = np.where(data[column].shift(-off) > data[column], 1,
                                                    np.where(data[column].shift(-off) < data[column], 0, 0))
        
    if drop_recent:  # Drop to remove NaNs
        data = drop_recent_with_offset(data, max(offset))

    return data




def create_event_for_crossing(data: pd.DataFrame,  src_column: str, target_column: str, decay_factor: float = 0.98) -> pd.DataFrame:
    """
    Create an event for when the target_column crosses the src_column.
    If the target_column crosses the src_column from below to above, the apply_event

    create_event_for_crossing(df, "close_bollinger_lower_30", "close") will create an event for when the close price crosses the lower bollinger band

    IF you want to create an event for when the close price crosses the upper bollinger band, you can use create_event_for_crossing(df, "close", "close_bollinger_upper_30")
    
    The src column should be the lower value and the target column should be the higher value

    """
    # Create a Dataframe of the events. The Event schema needs to be in this format [date, name, value, sentiment]
    events = pd.DataFrame(columns=["date", "name", "value", "sentiment"])
    event_name = f"crossed_{src_column}_{target_column}"

    # Loop through the data and check for the crossing. If the target_column crosses the src_column from below to above, add an event
    for i in range(1, len(data)):
        if data[src_column][i] > data[target_column][i] and data[src_column][i-1] < data[target_column][i-1]:
            
            temp_df = pd.DataFrame([[data["date"][i], "crossed", 1, 1]], columns=events.columns)
            events = pd.concat([events, temp_df], axis=0)

    data = apply_event(data, events, event_name, decay_factor=decay_factor)

    return data

In [ ]:
df = up_down(df, "close", 1)
# df

In [ ]:
df = apply_technical_indicators(df, ["open", "close", "high", "low", "volume"], [7, 14, 20, 30, 50, 100])
# df

In [ ]:
df = create_event_for_crossing(df, "close_bollinger_lower_20", "close")
df = create_event_for_crossing(df, "close", "close_bollinger_upper_20")
df

In [ ]:
df

In [ ]:
# %%capture
# df = percent_change(df, "close", [1, 7, 14, 30])
# df = future_value(df, "close", 1)
# df.tail(10)

In [ ]:
# df = apply_event(df, dividends, prefix_name="dividends", falloff=True, decay_factor=0.95)
# df = apply_event(df, splits, prefix_name="splits", falloff=True, decay_factor=0.95)
# df = is_not_nan_column_and_default(df, ["dividends_event_value", "dividends_event_sentiment"])
# df = is_not_nan_column_and_default(df, ["splits_event_value", "splits_event_sentiment"])
df

In [ ]:

df = create_lag_features(df, "open", lag_steps=list(range(1, 30)))
df = create_lag_features(df, "close", lag_steps=list(range(1, 30)))
df = create_lag_features(df, "high", lag_steps=list(range(1, 30)))
df = create_lag_features(df, "low", lag_steps=list(range(1, 30)))
df = create_lag_features(df, "volume", lag_steps=list(range(1, 30)))
# df = create_lag_features(df, "close_percent_change_1", lag_steps=[1, 2, 3, 4, 5, 6, 7, 10, 14, 21, 30, 60])

df = day_of_week(df)
df = day_of_month(df)
df = month(df)
df = year(df)
df = week_of_year(df)


# df = create_lag_features(df, "event_name", lag_steps=list(range(1, 60)))
df

In [ ]:
df = macd(df, "close", 50, 200, 9)

In [ ]:
df.filter(like= "open")

In [ ]:
df.to_parquet("../data/dataset.parquet")